In [1]:
import pandas as pd
from keras.utils import to_categorical

pd.set_option('display.max_rows', None)
import numpy  as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import keras
from tensorflow import keras
from keras.models import Model
from keras.metrics import Precision, Recall

import operator


import tensorflow as tf
from keras.losses import CategoricalCrossentropy
import segmentation_models as sm
from sklearn.metrics import precision_score, recall_score, f1_score
import wandb
from numpy import float16
from segmentation_models import Unet
from wandb.integration.keras import WandbCallback
import keras
# or from tensorflow import keras

keras.backend.set_image_data_format('channels_last')
# or keras.backend.set_image_data_format('channels_first')

Segmentation Models: using `keras` framework.


In [2]:
import os
import numpy as np
from sharedtask03.Model.helper import import_images
images=[]

path = '../data/01_train/train/'
folders = os.listdir(path)
i=True
for folder in folders:
    y_train,x_train=import_images(path+folder+'/')
    print(folder)


airplane
automobile
bird
cat
deer
dog
horse
ship
truck


In [4]:
x_train = tf.convert_to_tensor(x_train, dtype=float16)
y_train = tf.convert_to_tensor(y_train, dtype=float16)
# x_test = tf.convert_to_tensor(x_test, dtype=float16)
# y_test = tf.convert_to_tensor(y_test, dtype=float16)

# define number of channels
N = x_train.shape[-1]

# define model


In [8]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from sharedtask03.Model.Model_imported import create_res_net
from segmentation_models.losses import DiceLoss
from keras.optimizers import Adam

model = create_res_net() # or create_plain_net()
model.summary()
# save model after each epoch
cp_callback = ModelCheckpoint(
    filepath='ModelCheckpoint/',
    verbose=1
)
tensorboard_callback = TensorBoard(
    log_dir='tensorboard_logs/',
    histogram_freq=1
)



Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 batch_normalization_60 (BatchN  (None, 32, 32, 3)   12          ['input_5[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 conv2d_64 (Conv2D)             (None, 32, 32, 64)   1792        ['batch_normalization_60[0][0]'] 
                                                                                                  
 re_lu_58 (ReLU)                (None, 32, 32, 64)   0           ['conv2d_64[0][0]']        

In [99]:

path = '../data/01_train/train/'
train_ds = keras.utils.image_dataset_from_directory(path,
                                                    image_size=(32, 32),
                                                    color_mode='rgb',
                                                    seed=1234
                                                    ,validation_split=0.5
                                                    ,subset='training',
                                                     labels='inferred',
                                                    label_mode='categorical')

Found 45000 files belonging to 9 classes.
Using 22500 files for training.


In [106]:
from keras.layers import BatchNormalization
from keras.layers import Dense, Input, Flatten, Dropout, Embedding, Conv2D
from keras.layers import MaxPool2D

from keras.layers import UpSampling2D


def get_model():
        # input in the shape of a single digit
    input = keras.Input((32, 32,3))
    # reshape to match requirements of convolutional layer: (None, 28, 28) -> (None, 28, 28, 1)
    #expanded_input=tf.expand_dims(input, axis=-1)

    # Block 1: (None, 28, 28, 1) -> (None, 14, 14, 4)
    conv_1 = keras.layers.Conv2D(filters=4, kernel_size=3, padding='same')(input)
    conv_2 = keras.layers.Conv2D(filters=4, kernel_size=3, padding='same')(conv_1)
    max_pool_1 = keras.layers.MaxPool2D()(conv_2)

    # Block 2: (None, 14, 14, 4) -> (None, 7, 7, 4)
    conv_3 = keras.layers.Conv2D(filters=4, kernel_size=3,  padding='same')(max_pool_1)
    conv_4 = keras.layers.Conv2D(filters=4, kernel_size=3,  padding='same')(conv_3)
    # Residual connection
    res_1 = keras.layers.Add()([max_pool_1, conv_4])
    max_pool_2 = keras.layers.MaxPool2D()(res_1)

    # Block 3: (None, 7, 7, 4) -> (None, 7, 7, 4)
    conv_5 = keras.layers.Conv2D(filters=4, kernel_size=3,  padding='same')(max_pool_2)
    conv_6 = keras.layers.Conv2D(filters=4, kernel_size=3,  padding='same')(conv_5)
    # Residual connection
    res_2 = keras.layers.Add()([max_pool_2, conv_6])

    # (None, 7, 7, 4) -> (None, 5, 5, 8)
    conv_7 = keras.layers.Conv2D(filters=4, kernel_size=3, )(res_2)
    # (None, 5, 5, 8) -> (None, 3, 3, 16)
    conv_8 = keras.layers.Conv2D(filters=4, kernel_size=3, )(conv_7)
    # (None, 3, 3, 16) -> (None, 1, 1, 16)
    conv_9 = keras.layers.Conv2D(filters=4, kernel_size=3, )(conv_8)

    # flatten input into a single dimension: (None, 1, 1, 16) -> (None, 16)
    flat = keras.layers.Flatten()(conv_9)

    # apply dense layer for classification: (None, 16) -> (None, 10)
    output = keras.layers.Dense(units=9, activation='softmax')(flat)
    # build and compile model
    model = keras.Model(input, output)
    return model

#model.summary()

In [107]:

model = get_model()
#model.summary()
print('Number of training batches: %d' % tf.data.experimental.cardinality(train_ds).numpy())

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['categorical_accuracy'])

model.fit(train_ds,batch_size=100,epochs=100
          #,
          #validation_data=(train_2D_x[1000:],train_2D_y[1000:])
)

Number of training batches: 704
Epoch 1/100
704/704 [==============================] - 24s 31ms/step - loss: 5.1013 - categorical_accuracy: 0.2612
Epoch 2/100
704/704 [==============================] - 22s 31ms/step - loss: 1.6822 - categorical_accuracy: 0.3707
Epoch 3/100
704/704 [==============================] - 23s 32ms/step - loss: 1.6092 - categorical_accuracy: 0.4016
Epoch 4/100
704/704 [==============================] - 23s 32ms/step - loss: 1.5473 - categorical_accuracy: 0.4247
Epoch 5/100
704/704 [==============================] - 22s 31ms/step - loss: 1.4883 - categorical_accuracy: 0.4535
Epoch 6/100
704/704 [==============================] - 21s 29ms/step - loss: 1.4545 - categorical_accuracy: 0.4677
Epoch 7/100
704/704 [==============================] - 21s 30ms/step - loss: 1.4332 - categorical_accuracy: 0.4747
Epoch 8/100
704/704 [==============================] - 23s 32ms/step - loss: 1.4160 - categorical_accuracy: 0.4827
Epoch 9/100
704/704 [===========================


KeyboardInterrupt



In [91]:

def get_simpel_model():
        # input in the shape of a single digit
    input = keras.Input((32, 32,3))
    flat = keras.layers.Flatten()(input)
    # reshape to match requirements of convolutional layer: (None, 28, 28) -> (None, 28, 28, 1)
    #expanded_input=tf.expand_dims(input, axis=-1)
    # apply dense layer for classification: (None, 16) -> (None, 10)
    output = keras.layers.Dense(units=1, activation='softmax')(flat)
    # build and compile model
    model = keras.Model(input, output)
    return model


In [96]:
model = get_simpel_model()
model.summary()
print('Number of training batches: %d' % tf.data.experimental.cardinality(train_ds).numpy())

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['categorical_accuracy'])

model.fit(train_ds,batch_size=9,epochs=100
          #,
          #validation_data=(train_2D_x[1000:],train_2D_y[1000:])
)

Model: "model_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_45 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 flatten_21 (Flatten)        (None, 3072)              0         
                                                                 
 dense_40 (Dense)            (None, 1)                 3073      
                                                                 
Total params: 3,073
Trainable params: 3,073
Non-trainable params: 0
_________________________________________________________________
Number of training batches: 704
Epoch 1/100
704/704 [==============================] - 6s 8ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 2/100
704/704 [==============================] - 5s 7ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
Epoch 3/100
124/704 [====>.........................] -


KeyboardInterrupt



In [51]:

predictions = np.array([])
labels =  np.array([])
for x, y in train_ds:
  predictions = np.concatenate([predictions, model.predict_classes(x)])
  labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

tf.math.confusion_matrix(labels=labels, predictions=predictions).numpy()

AttributeError: 'Functional' object has no attribute 'predict_classes'

In [ ]:
from keras.metrics import MeanIoU, OneHotMeanIoU
#from sklearn.metrics import recall_score, f1_score, precision_score
from keras import metrics

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001),
              metrics = ['accuracy','categorical_accuracy',OneHotMeanIoU (num_classes=5)
                                   ]
              # metrics = ['accuracy','sparse_categorical_accuracy','categorical_accuracy','categorical_crossentropy',#MeanIoU(num_classes=5)
              #                     ]
              )

#tf.keras.metrics.MeanIoU(num_classes=2)]


In [ ]:
def load_model:
    input = Input(shape=(32,32,3))
    norm1 = BatchNormalization()(input)

    irnn = component_loop(norm1)

In [9]:
model.fit(
    x=x_train,
    y=y_train,
    epochs=20,
    verbose=1,
    #validation_data=(x_test, y_test),
    batch_size=128,
    #callbacks=[cp_callback, tensorboard_callback]
)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\users\vdwti\pycharmprojects\shared-tasks-wintersemester-2022-23\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\users\vdwti\pycharmprojects\shared-tasks-wintersemester-2022-23\venv\lib\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "c:\users\vdwti\pycharmprojects\shared-tasks-wintersemester-2022-23\venv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\users\vdwti\pyc

In [ ]:


# Configs
batch_size = 17
epochs = 300
metrics = ['categorical_accuracy']
label_mapping = 'Ohne_Auto_See'  # Alternative:'Grünflächen'

match label_mapping:
    case 'Ohne_Auto_See':
        # labels = {0: 'None', 1: 'Wiese', 2: 'Straße', 3: 'Schienen', 4: 'Haus', 5: 'Wald'}
        labels = ['None', 'Wiese', 'Straße', 'Schienen', 'Haus', 'Wald']
    case 'Grünflächen':
        # labels = {0: 'None', 1: 'Grünflächen', 2: 'Straße', 3: 'Schienen', 4: 'Haus'}
        labels = ['None', 'Grünflächen', 'Straße', 'Schienen', 'Haus']
    case _:  # else
        labels = [0, 1, 2, 3, 4, 5, 6, 7, 8]

""" wird aktuell nicht mehr benötigt. Bitte stehen lassen
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
# gpus = tf.config.list_physical_devices('GPU')
# tf.config.set_visible_devices(gpus[0], 'GPU')
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
"""


def cnn_sweep():
    wandb.init()
    # Access all hyperparameter values through wandb.config
    config = wandb.config
    # set configs
    optim = tf.keras.optimizers.Adam(config.learning_rate)

    loss = JaccardLoss()  # to avoid errors
    match config.loss:
        case 'JaccardLoss':
            loss = JaccardLoss()
        case 'DiceLoss':
            loss = DiceLoss()
        case 'CategoricalCrossentropy':
            loss = CategoricalCrossentropy()

    """wird aktuell nicht mehr benötigt. Bitte stehen lassen
    import wandb
    # login
    run = wandb.init(project="cnn_segmentation_models", entity="pds_project", name='segmentation_models_Unet',
                     )
    run.config.update({"epochs": epochs, "batch_size": batch_size, 'metrics': metrics, 'loss': loss, 'optim': optim,
                       'learning_rate': learning_rate,
                       'backbone_name': backbone_name,
                       'activation': activation,
                       'encoder_freeze': encoder_freeze,
                       'augmentation': augmentation
                       })
    run.config.update({'images_train': len(x_train_data),
                       'images_test': len(x_test_data),
                       })
    """
    # wandb.log({"label_mapping": str(label_mapping)})

    # Choose bands
    bands = list(range(0, 104, config.band_dist))
    bands.append(105)
    bands.append(106)
    bands.append(107)
    bands.append(108)

    # import Data
    x, y = import_labeled_photos(bands=bands, label_mapping=label_mapping)

    x_train = x[0:22]
    x_train.append(x[29:])
    y_train = y[0:22]
    y_train.append(y[29:])

    x_test = x[22:29]
    y_test = y[22:29]
    del x, y

    # add augmentations
    import albumentations as A

    transform_light = A.Compose([
        A.HorizontalFlip(p=1),
        A.RandomBrightnessContrast(p=1, brightness_by_max=False,
                                   contrast_limit=(-config.aug_contrast_light, config.aug_contrast_light),
                                   brightness_limit=(-config.aug_brightness_light, config.aug_brightness_light)),
        A.MultiplicativeNoise(p=1, multiplier=(1 - config.aug_noise_light, 1 + config.aug_noise_light),
                              elementwise=False),
        A.PixelDropout(p=1, dropout_prob=config.aug_dropout_light)
    ])
    transform_middle = A.Compose([
        A.VerticalFlip(p=1),
        A.RandomBrightnessContrast(p=1, brightness_by_max=False,
                                   contrast_limit=(-config.aug_contrast_middle, config.aug_contrast_middle),
                                   brightness_limit=(-config.aug_brightness_middle, config.aug_brightness_middle)),
        A.MultiplicativeNoise(p=1, multiplier=(1 - config.aug_noise_middle, 1 + config.aug_noise_middle),
                              elementwise=False),
        A.GaussianBlur(p=1),
        A.PixelDropout(p=1, dropout_prob=config.aug_dropout_middle)
    ])
    transform_hard = A.Compose([
        A.RandomCrop(width=150, height=150),
        # A.ElasticTransform(p=1),
        A.GaussianBlur(p=1),
        A.RandomBrightnessContrast(p=1, brightness_by_max=False,
                                   contrast_limit=(-config.aug_contrast_hard, config.aug_contrast_hard),
                                   brightness_limit=(-config.aug_brightness_hard, config.aug_brightness_hard)),
        A.MultiplicativeNoise(p=1, multiplier=(1 - config.aug_noise_hard,
                                               1 + config.aug_noise_hard), elementwise=True),
        A.PixelDropout(p=1, dropout_prob=config.aug_dropout_hard),
        A.PadIfNeeded(min_height=224, min_width=224, p=1)
    ])

    aug_list = [transform_light, transform_middle, transform_hard]

    pict_with_labels = zip(x_train.copy(), y_train.copy())
    for image, mask in pict_with_labels:
        for aug in aug_list:
            transformed = aug(image=image, mask=mask)
            x_train.append(transformed['image'])
            y_train.append(transformed['mask'])

    # Transform Data to tensor
    x_train = tf.convert_to_tensor(x_train, dtype=float16)
    y_train = tf.convert_to_tensor(y_train, dtype=float16)
    x_test = tf.convert_to_tensor(x_test, dtype=float16)
    y_test = tf.convert_to_tensor(y_test, dtype=float16)

    # define number of channels
    N = x_train.shape[-1]

    # define model
    model = Unet(backbone_name=config.backbone, encoder_freeze=config.encoder_freeze, encoder_weights=None,
                 input_shape=(None, None, N), activation=config.activation,
                 classes=len(labels))

    # model.summary()
    model.compile(optim, loss=loss, metrics=metrics)

    # train
    model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,  # config.batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test),
        callbacks=[WandbCallback()]
    )
    # make predictions train
    y_train = tf.reshape(tf.math.argmax(y_train, axis=3), [-1]).numpy()
    pred_y_train = model.predict(x_train)
    pred_y_train = tf.reshape(tf.math.argmax(pred_y_train, axis=3), [-1]).numpy()

    # make predictions test
    y_test = tf.reshape(tf.math.argmax(y_test, axis=3), [-1]).numpy()
    pred_y_test = model.predict(x_test)
    pred_y_test = tf.reshape(tf.math.argmax(pred_y_test, axis=3), [-1]).numpy()

    labels_train = np.unique(np.concatenate((y_train, pred_y_train)))
    labels_test = np.unique(np.concatenate((y_test, pred_y_test)))

    # calculate and log metrics
    precision_train = precision_score(y_train, pred_y_train, labels=labels_train, average='micro')
    precision_test = precision_score(y_test, pred_y_test, labels=labels_test, average='micro')

    recall_train = recall_score(y_train, pred_y_train, labels=labels_train, average='micro')
    recall_test = recall_score(y_test, pred_y_test, labels=labels_test, average='micro')

    f1_train = f1_score(y_train, pred_y_train, labels=labels_train, average='micro')
    f1_test = f1_score(y_test, pred_y_test, labels=labels_test, average='micro')

    labels_test_str = operator.itemgetter(*labels_test)(labels)
    wandb.log(
        {"conf_mat": wandb.plot.confusion_matrix(probs=None, y_true=y_test, preds=pred_y_test,
                                                 class_names=labels_test_str)})

    wandb.log({'precision_train': precision_train})
    wandb.log({'precision_test': precision_test})

    wandb.log({'recall_train': recall_train})
    wandb.log({'recall_test': recall_test})

    wandb.log({'f1_train': f1_train})
    wandb.log({'f1_test': f1_test})
    print("Finshed Job")
    wandb.finish()

if __name__ == '__main__':
    """
    Better use Sweep_upload_data.ipynb to avoid errors and bad visualisation
    """

    # define sweep_id
    sweep_id = 'ovzbvxa7'
    # wandb sweep sweep.yaml

    # run the sweep
    wandb.agent(sweep_id, function=cnn_sweep, project="cnn_segmentation_models", entity="pds_project")


In [1]:
from sharedtask02.Model.model import load_data, create_traindataset


train, sampleTest, sampleSubmission = load_data()
x_train, y_train, x_test, y_test = create_traindataset(train)

In [42]:
# Read in the csv data using pandas
def load_data():
    train = pd.read_csv('../data/01_train/train.csv',header=None)
    sampleTest   = pd.read_csv('../data/01_train/sampleTest.csv')
    sampleSubmission  = pd.read_csv('../data/01_train/sampleSubmission.csv')
    return train,sampleTest,sampleSubmission

In [43]:
#Feature Scaling
# load data
train, sampleTest, sampleSubmission = load_data()

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
train_scaled = sc.fit_transform(train)

In [44]:
x = []
y = []
n_future = 7 # next 4 days temperature forecast
n_past = 90 # Past 90 days
for j in range(0,len(train_scaled)):
    for i in range(0,len(train_scaled[j])-n_past-n_future+1):
        x.append(train_scaled[j,i : i + n_past ])
        y.append(train_scaled[j,i + n_past : i + n_past + n_future ])
x , y = np.array(x), np.array(y)
x = np.reshape(x, (x.shape[0] , x.shape[1], 1) )


x_train = []
y_train = []
n_future = 7 # next 4 days temperature forecast
n_past = 90 # Past 30 days
n_overlap=3
overlap=[]
for i in range(1,n_overlap+1):
    temp=int(n_past-(n_past/n_overlap)*(i))
    overlap.append(temp)

for j in range(0,len(train_scaled)):
    for i in range(0,(len(train_scaled[j])-n_past-n_future+1)):
        for o in overlap:
            von=i+o
            bis=i+o + n_past
            if(len(train_scaled[j,von : bis ])==90 and len(train_scaled[j,bis : bis + n_future ])==7):
                x_train.append(train_scaled[j,von : bis ])
                y_train.append(train_scaled[j,bis : bis + n_future ])

#Ah = np.vstack(x_train)
#Av = np.hstack(x_train)
#x_trainl=x_train
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )
x=x_train

In [45]:
from keras.models import Sequential
from keras.layers import LSTM,Dense ,Dropout
# Fitting RNN to training set using Keras Callbacks. Read Keras callbacks docs for more info.

In [46]:
from keras.layers import Bidirectional, RNN, SimpleRNN


def load_model(x_train):
    model = Sequential()
    model.add(Bidirectional(SimpleRNN(units=2, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
    model.add(Dropout(0.2))
    model.add(LSTM(units= 2 , return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units= 2 , return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(units= 2))
    model.add(Dropout(0.2))
    model.add(Dense(units = n_future,activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc','mean_squared_error'])
    return model


In [ ]:
# import model_selection module of scikit-learn
from sklearn import model_selection
# initiate the k-fold class from model_selection module
splits=2
#for i in range(splits):
kf = model_selection.KFold(n_splits=splits,shuffle=True)
for fold, (trn_, val_) in enumerate(kf.split(X=x)):
    #split dataset
    x_train=x[trn_]
    y_train=y[trn_]
    x_test=x[val_]
    y_test=y[val_]

    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)

    model=load_model(x_train)
    model.fit(x_train, y_train, epochs=200,batch_size=256,validation_data=(x_test, y_test))

(31986, 90, 1)
(31986, 7)
(31986, 90, 1)
(31986, 7)
Epoch 1/200
 22/125 [====>.........................] - ETA: 7s - loss: 0.4113 - acc: 0.1474 - mean_squared_error: 0.4113

KeyboardInterrupt: 

In [48]:
predicted_temperature = model.predict(x_test)
predicted_temperature = sc.inverse_transform(predicted_temperature)
#predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))


KeyboardInterrupt



In [ ]:
y_test

In [ ]:
predicted_temperature

In [ ]:
dif=y_test-predicted_temperature
dif

In [ ]:
from numpy import mean

mean(abs(dif))